In [1]:
import pymongo

In [2]:
import os
import pymysql
import pandas as pd
password = os.environ.get('MYSQL_PASSWORD') 

In [3]:
try:
    con = pymysql.connect(
                            host='localhost',
                            user='root',
                            password=password,
                            charset='utf8mb4'
                            )
    print("Connection to the database was successful!")
except pymysql.Error as e:
    print(f"An error occurred while connecting to the database: {e}")

Connection to the database was successful!


In [4]:
# Some useful functions: 

# craete a MySQL database 
def database_creator(database_name):
    
    conn = connector(database=None)
    
    # Create a cursor object
    cursor = conn.cursor()

    cursor.execute(f"DROP DATABASE IF EXISTS {database_name}")
    
    # Execute the CREATE DATABASE SQL command
    cursor.execute(f"CREATE DATABASE {database_name}")

    # Commit the transaction
    conn.commit()

    # Close the connection
    conn.close()

# Connect to MySQL
def connector (database, password=password, host='localhost', 
               user='root', charset='utf8mb4',):
    
    conn = pymysql.connect(
                        host=host,
                        user=user,
                        password=password,
                        charset=charset, 
                        database=database)
    
    return conn

def table_names (database):
    
    con = connector(database)
    
    query = "SELECT table_name FROM information_schema.tables WHERE table_schema='{}';".format(database)

    # Execute the query and store the results in a Pandas DataFrame
    tables = pd.read_sql_query(query, con)

    # Print the list of tables
    return tables

def delete_database(database):
    conn = connector(database=None)
    cursor = conn.cursor()

    # Replace <database_name> with the name of the database you want to delete
    database_name = database

    # Execute the DROP DATABASE SQL command
    cursor.execute(f"DROP DATABASE {database_name}")

    # Commit the transaction
    conn.commit()

    # Close the connection
    conn.close()
    
def database_names():

    # Create a cursor object
    cursor = con.cursor()

    # Execute the SHOW DATABASES command
    cursor.execute('SHOW DATABASES')

    # Fetch all the results as a list of tuples
    results = cursor.fetchall()

    # Print the names of all databases in the MySQL server
    for row in results:
        print(row[0])

    # Close the cursor and connection
    cursor.close()
    con.close()

In [6]:
# Connect to MongoDB
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
mongo_db = mongo_client["customers"]
mongo_collection = mongo_db["information"]

In [12]:
database_creator ('customers')

In [19]:
con = connector('customers')

In [22]:
# Define MySQL table schema
mysql_cursor = con.cursor()

mysql_cursor.execute('''
    CREATE TABLE IF NOT EXISTS information (
        id INT AUTO_INCREMENT PRIMARY KEY,
        first_name VARCHAR(255),
        last_name VARCHAR(255),
        country_of_residence VARCHAR(255),
        date_of_birth DATE,
        email VARCHAR(255),
        state_of_residence VARCHAR(255),
        postal_code VARCHAR(255)
    )
''')


0

In [23]:
# Extract data from MongoDB and insert into MySQL table
for document in mongo_db["information"].find():
    user_data = (
        document.get("id"),
        document.get("first_name"),
        document.get("last_name"), 
        document.get("country_of_residence"),
        document.get("date_of_birth"),
        document.get("email"),
        document.get("state_of_residence"),
        document.get("postal_code")
    )
    mysql_cursor.execute('''
        INSERT INTO information (id, first_name, last_name, country_of_residence, date_of_birth, email, state_of_residence, postal_code)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    ''', user_data)


In [16]:
database_names()

customers
information_schema
mysql
performance_schema
sakila
sql_hr
sql_inventory
sql_invoicing
sql_store
sys
world


In [ ]:
delete_database('customers')

In [25]:
table_names('customers')

C:\Users\dania\AppData\Local\Temp\ipykernel_26216\30147618.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tables = pd.read_sql_query(query, con)


,TABLE_NAME
0,information


In [24]:
pd.read_sql("""
SELECT *
FROM information """, con)

C:\Users\dania\AppData\Local\Temp\ipykernel_26216\3924582339.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql("""


,id,first_name,last_name,country_of_residence,date_of_birth,email,state_of_residence,postal_code
0,1,John,Smith,USA,1980-01-01,john.smith@gmail.com,CA,AABBCC
1,2,Jim,Carter,Canada,1970-01-01,jimi.carter@yahoo.com,AB,ZZYYWW
2,3,Sarah,Elton,USA,1940-01-01,selton@gmail.com,USA,FFGGHH
3,4,Kim,Herford,USA,1987-01-01,kim.h@gmail.com,TX,KKJJHH
